# Constructing NEDD for Drug-Disease Prediction

## Packages

In [1]:
# coding=utf-8

In [2]:
from __future__ import print_function
from __future__ import division

In [3]:
import sys
import os
import tempfile
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

from hin2vec.ds import network

## Functions

In [4]:
def load_a_HIN_from_pandas(edges, print_graph=False):
    """
    edges = list(pd.df)
    """

    def reverse(df):
        """
        reverse source & dest
        """
        df = df.rename({'source_node': 'dest_node', 'dest_node': 'source_node',
                        'source_class': 'dest_class', 'dest_class': 'source_class'},
                       axis=1)
        # reverse edge_class
        df.edge_class = df.edge_class.map(lambda x: '-'.join(reversed(x.split('-'))))
        return df

    print('load graph from edges...')
    g = network.HIN()
    if isinstance(edges, list):
        edges = pd.concat(edges, sort=False)
    edges = edges.append(reverse(edges), sort=False, ignore_index=True)

    for index, row in edges.iterrows():
        g.add_edge(row['source_node'], row['source_class'],
                   row['dest_node'], row['dest_class'], row['edge_class'],
                   row['weight'])
    if print_graph:
        g.print_statistics()
    print('finish loading graph!')
    return g

In [5]:
def getVec(edges, options, node_vec_fname, path_vec_fname=None): # , path_vec_fname):

    def output_node2vec(_g, _tmp_node_vec_fname, _node_vec_fname):
        if os.stat(_tmp_node_vec_fname).st_size == 0:
            print("Temp file is empty")

        with open(_tmp_node_vec_fname) as _f:
            with open(_node_vec_fname, 'w') as fo:
                id2node = dict([(v, k) for k, v in _g.node2id.items()])
                first = True
                for line in _f:
                    if first:
                        first = False
                        fo.write(line)
                        continue

                    id_, vectors = line.strip().split(' ', 1)
                    line = '%s %s\n' % (id2node[int(id_)], vectors)
                    fo.write(line)

    def output_path2vec(g, tmp_path_vec_fname, path_vec_fname):
        with open(tmp_path_vec_fname) as f:
            with open(path_vec_fname, 'w') as fo:
                fo.write(f.read())
                return

    g = load_a_HIN_from_pandas(edges, True)

    print('Generate random walks...')
    _, tmp_walk_fname = tempfile.mkstemp()
    print('walk file', tmp_walk_fname)
    # tmp_walk_fname = 'temp_walk'
    with open(tmp_walk_fname, 'w') as f:
        for walk in g.random_walks(options.walk_num, options.walk_length):
            f.write('%s\n' % ' '.join(map(str, walk)))

    _, tmp_node_vec_fname = tempfile.mkstemp()
    print('tmp_node_vec_fname file', tmp_node_vec_fname)
    _, tmp_path_vec_fname = tempfile.mkstemp()  # useless
    print('walk tmp_path_vec_fname', tmp_path_vec_fname)

    # if a temp file already exists then replace it with an empty file
    if os.path.exists(tmp_node_vec_fname):
        open(tmp_node_vec_fname, 'w').close()

    if os.path.exists(tmp_path_vec_fname):
        open(tmp_path_vec_fname, 'w').close()

    print('Learn representations...')
    statement = ("hin2vec/model_c/bin/hin2vec -size %d -train %s -alpha %f "
                 "-output %s -output_mp %s -window %d -negative %d "
                 "-threads %d -no_circle %d -sigmoid_reg %d "
                 "" % (options.dim,
                       tmp_walk_fname,
                       options.alpha,
                       tmp_node_vec_fname,
                       tmp_path_vec_fname,
                       options.window,
                       options.neg,
                       options.num_processes,
                       1 - (options.allow_circle * 1),
                       options.sigmoid_reg * 1))
    print(statement)
    os.system(statement)

    print('Dump vectors...')
    output_node2vec(g, tmp_node_vec_fname, node_vec_fname)
    
    if path_vec_fname:
        output_path2vec(g, tmp_path_vec_fname, path_vec_fname)

    del g

    return 0

In [6]:
def predict(drug, disease, edgeDiDr, randomForestOption=None):

    defaultRandomForestOption = {
        'max_depth': 25,
        'n_estimators': 300,
        'min_samples_split': 2,
        'oob_score': True,
        'min_impurity_decrease': 0.0,
        'max_leaf_nodes': None,
        'criterion': 'gini',
        'min_samples_leaf': 1,
        'max_features': 'auto',
        'n_jobs': -1
    }

    # random forest
    if randomForestOption:
        for k in randomForestOption:
            assert k in defaultRandomForestOption
            defaultRandomForestOption[k] = randomForestOption[k]

    print('start testing node vectors')
    all_list = []

    for j in range(len(disease)):
        drug_ = drug.copy()
        values = drug_.to_numpy(copy=False)
        values *= disease.values[j] 
        drug_.insert(0, 'disease', disease.index[j])
        all_list.append(drug_)

    all_df = pd.concat(all_list).reset_index().rename({0: 'drug'}, axis=1)
    
    all_df = (
        all_df.merge(
            edgeDiDr.loc[:, ['dest_node', 'source_node', 'weight']], how='outer',
            left_on=['drug', 'disease'], right_on=['dest_node', 'source_node'])
            .drop(['dest_node', 'source_node'], axis=1)
            .fillna(0))
    assert edgeDiDr.shape[0] == sum(all_df.weight)
    all_df = all_df.loc[:,
             ['disease', 'drug', 'weight'] + list(range(1, all_df.shape[1] - 2))]


    class1 = all_df[all_df.weight == 1]
    class2 = all_df[all_df.weight == 0]

    class2_train = class2.sample(int(class1.shape[0] * 17))
    train_data = pd.concat([class1, class2_train])
    y = train_data.weight.to_numpy()
    X = train_data.iloc[:, 3:].to_numpy()

    print('start training random forest classifier')
    rnd_clf = Pipeline([
        ("scaler", StandardScaler()),
        ('random forest',
         RandomForestClassifier(**defaultRandomForestOption))
    ])

    rnd_clf.fit(X, y)

    y_pred_rf_test_score = rnd_clf.predict_proba(all_df.iloc[:, 3:].to_numpy())

    res = pd.DataFrame(all_df.iloc[:,:2]).assign(predict_score=y_pred_rf_test_score[:, 1])

    return res

def calculate_accuracy(edges, pred, threshold):

    preds = pred[["disease", "drug", "predict_score"]]
    preds['predict_score'] = preds['predict_score'].astype(float)
    preds['predict_score'] = preds['predict_score'].apply(lambda x: 1 if x > threshold else 0)

    act = edges[["source_node", "dest_node", "weight"]]
    act = act.rename(columns={'source_node': 'disease', 'dest_node': 'drug', 'weight':'predict_score'})


    pred_in = preds[preds['predict_score'] == 1]
    pred_out = preds[preds['predict_score'] == 0]

    corr_pred_in = pd.merge(edges, pred_in, how='inner', left_on=['source_node', 'dest_node', 'weight'], right_on=['disease', 'drug', 'predict_score'])
    corr_pred_out = pd.merge(edges, pred_out, how='inner', left_on=['source_node', 'dest_node', 'weight'], right_on=['disease', 'drug', 'predict_score'])

    total_edges = edges.shape[0]

    acc = (corr_pred_in.shape[0] + corr_pred_out.shape[0]) / act.shape[0]

    return acc

def calculate_acc_and_stdev(edges, pred, threshold, n_splits=5):
    kf = KFold(n_splits=n_splits)
    
    accuracy_list = []
    for train_index, test_index in kf.split(edges):
        train_edges = edges.iloc[train_index]
        test_edges = edges.iloc[test_index]
        
        acc = calculate_accuracy(train_edges, pred, threshold)
        accuracy_list.append(acc)
    
    accuracy_std = np.std(accuracy_list)
    return acc, accuracy_std

In [7]:
def thFilter(edge, th, n=1):

    def func(grp):
        res = grp.query("weight>=%f" % th)
        if len(res) < n:
            return grp.nlargest(n, 'weight')
        return res

    a = edge.reset_index()
    b = pd.concat([a.loc[:, ['index', 'dest_node', 'weight']].rename({'dest_node': 'node'}, axis=1),
                   a.loc[:, ['index', 'source_node', 'weight']].rename({'source_node': 'node'}, axis=1)])
    c = b.groupby('node').apply(func)
    d = c.loc[:, 'index'].unique()
    return edge.loc[d, :]

In [8]:
class Options:
    def __init__(self):
        """
        this is the default setting of options
        """
        # The threshold for disease similarity scores
        self.th_di = 0.7
        # The threshold for drug similarity scores
        self.th_dr = 0.8
        # The length of each random walk
        self.walk_length = 1000
        # The number of random walks starting from each node
        self.walk_num = 10
        # Number of negative examples (>0) for negative sampling, 0 for hierarchical softmax
        self.neg = 5
        # Dimensionality of word embeddings
        self.dim = 90
        # Starting learning rate
        self.alpha = 0.025
        # Max window length
        self.window = 6
        # Number of processes
        self.num_processes = 4
        # Set to all circles in relationships between nodes
        self.allow_circle = False
        # Use sigmoid function for regularization for meta-path vectors
        self.sigmoid_reg = False
        # Random forest option
        self.randomForestOption = {
            'max_depth': 25,
            'n_estimators': 300,
            'min_samples_split': 2,
            'oob_score': True,
            'min_impurity_decrease': 0.0,
            'max_leaf_nodes': None,
            'criterion': 'gini',
            'min_samples_leaf': 1,
            'max_features': 'auto',
            'n_jobs': -1
        }

## Training

In [9]:
# set NEDD parameters

options = Options()
# The threshold for disease similarity scores
options.th_di = 0.7
# The threshold for drug similarity scores
options.th_dr = 0.8
# The length of each random walk
options.walk_length = 1000
# The number of random walks starting from each node
options.walk_num = 10
# Number of negative examples (>0) for negative sampling, 0 for hierarchical softmax
options.neg = 5
# Dimensionality of word embeddings
options.dim = 90
# Starting learning rate
options.alpha = 0.025
# Max window length
options.window = 6
# Number of processes
options.num_processes = 4
# Set to all circles in relationships between nodes
options.allow_circle = False
# Use sigmoid function for regularization for meta-path vectors
options.sigmoid_reg = False
# Random forest option
options.randomForestOption = {
            'max_depth': 25,
            'n_estimators': 300,
            'min_samples_split': 2,
            'oob_score': True,
            'min_impurity_decrease': 0.0,
            'max_leaf_nodes': None,
            'criterion': 'gini',
            'min_samples_leaf': 1,
            'max_features': 'auto',
            'n_jobs': -1
        }

In [10]:
print('load edges...')

edgeDiDr = pd.read_csv('./Dataset/DCh-Miner_miner-disease-chemical_DiDr.csv', index_col=0)
edgeDiSim = pd.read_csv('./Dataset/DCh-Miner_miner-disease-chemical_DiSim.csv', index_col=0)
edgeDrSim = pd.read_csv('./Dataset/DCh-Miner_miner-disease-chemical_DrSim.csv', index_col=0)

print('finished loading edges!')

edgeDiSim_filtered = thFilter(edgeDiSim, th=options.th_di)
edgeDrSim_filtered = thFilter(edgeDrSim, th=options.th_dr)

print('finished filtering edges')

edges = pd.concat([edgeDiDr, edgeDrSim_filtered, edgeDiSim_filtered], sort=False)

nodeVec = 'nodeVec.txt'
print('nodevec file', nodeVec)

getVec(edges, options, nodeVec)

load edges...
finished loading edges!
finished filtering edges
nodevec file nodeVec.txt
load graph from edges...


/var/folders/x9/9l0xjq0d3qq2b6dqmt7vb_qr0000gn/T/ipykernel_22553/3409088710.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  edges = edges.append(reverse(edges), sort=False, ignore_index=True)


Di:
313
Dr:
593
{'Di-Dr': 0, 'Dr-Dr': 1, 'Di-Di': 2, 'Dr-Di': 3}
1
9
10
6
240
4
297
7
474
12
552
9
685
4
695
8
882
5
828
3
329
9
222
9
485
3
146
18
391
4
221
8
404
3
599
10
766
9
778
2
819
4
824
9
853
7
232
8
9
25
282
4
371
9
468
9
543
5
791
11
827
6
6
5
5
8
8
2
121
2
134
2
340
8
358
4
438
3
572
9
578
5
675
6
696
5
726
2
877
5
33
30
218
12
285
4
359
4
366
6
387
4
434
7
447
24
452
9
489
8
523
4
680
2
701
2
709
18
719
3
811
4
829
3
841
4
857
4
4
12
415
7
12
6
330
8
495
24
538
13
397
3
78
6
493
6
545
4
648
5
903
5
50
4
509
2
781
3
102
3
533
5
36
22
17
6
22
3
712
9
19
3
235
6
236
4
237
2
238
2
16
4
373
8
428
7
21
4
20
4
24
3
484
17
679
14
704
10
845
23
737
4
527
4
436
3
558
2
775
2
879
4
541
4
27
2
163
20
278
10
29
3
71
4
187
8
249
10
323
3
444
5
470
3
510
6
570
3
589
12
604
18
619
7
624
4
657
2
660
14
758
4
798
13
822
12
866
3
869
3
606
3
266
3
32
2
108
12
34
7
123
9
251
6
253
5
263
13
374
4
389
12
405
6
442
6
458
7
460
6
465
10
476
10
547
8
568
7
601
15
616
7
635
8
673
3
687
10
702
6
722

sh: hin2vec/model_c/bin/hin2vec: cannot execute binary file


0

In [11]:
# load embedding file
embed = pd.read_csv("nodeVec_pretrained.txt", sep=' ', header=None, skiprows=1)
drug = embed[embed[0].str.match('^DB\d+$')]
drug.set_index(0, inplace=True)
disease = embed[embed[0].str.match('^OMIM\d+$')]
disease.set_index(0, inplace=True)

res = predict(drug, disease, edgeDiDr, randomForestOption=options.randomForestOption)

res.to_csv('output.csv')

print('finished')
acc, stdev = calculate_acc_and_stdev(edgeDiDr, res, 0.5, 4)
print("Accuracy: " + str(acc))
print("StDev: " + str(stdev))

start testing node vectors
start training random forest classifier
finished
Accuracy: 0.9586206896551724
StDev: 0.0021537170474475536
